# Trabajo Practico N°1 NLP
Integrantes: 
1. Rodriguez y Barros, Francisco : R-4559/4
2. Slepoy, David : S-5782/7

### Importamos las librerias

In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\david\OneDrive\Escritorio\T.U.I.A\4.2 PROCESAMIENTO DEL LENGUAJE NATURAL\directorio\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Cargamos el modelo preentrenado SentenceTransformer para generar embeddings a partir de texto en múltiples idiomas
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Carga de datasets con Embeddings

In [3]:
# Cargamos los datasets con los embeddings ya previamente generados
# Dataset de juegos de mesa
df_juegos_de_mesa = pd.read_csv('./datasets_embeddings/df_juegos_de_mesa_emb.csv')
# Dataset de películas
df_peliculas = pd.read_csv('./datasets_embeddings/df_peliculas_emb.csv')
# Dataset de libros
df_libros = pd.read_csv('./datasets_embeddings/df_libros_emb.csv')


# Convertimos el contenido de la columna embeddings de string a arrays de NumPy en cada dataset para luego poder hacer las comparaciones
# df_libros
df_libros['embeddings'] = df_libros['embeddings'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))
# df_libros['embeddings'].apply(type).unique(), df_libros['embeddings'].head()

# df_peliculas
df_peliculas['embeddings'] = df_peliculas['embeddings'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))
# df_peliculas['embeddings'].apply(type).unique(), df_peliculas['embeddings'].head()

# df_juegos_de_mesa
df_juegos_de_mesa['embeddings'] = df_juegos_de_mesa['embeddings'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))
# df_juegos_de_mesa['embeddings'].apply(type).unique(), df_juegos_de_mesa['embeddings'].head()

In [4]:
print(df_peliculas['embeddings'].apply(type).unique())
print(df_libros['embeddings'].apply(type).unique())
print(df_juegos_de_mesa['embeddings'].apply(type).unique())

[<class 'numpy.ndarray'>]
[<class 'numpy.ndarray'>]
[<class 'numpy.ndarray'>]


# Clasificador de Estado de Ánimo

### Creamos el data set con los estados de animo para entrenar el modelo de regresion logistica

In [5]:
data = {
    "frase": [
        # 50 frases para "Alegre"
        "Feliz", "Contento", "Emocionado", "Entusiasmado", "Optimista",
        "Hoy es un día perfecto", "Me siento lleno de energía", "Todo está saliendo bien",
        "Hoy logré todo lo que quería", "El sol brilla y mi ánimo también",
        "Estoy disfrutando cada momento", "Qué día tan increíble", "Me siento agradecido por todo",
        "Todo es tan hermoso hoy", "Me encanta la vida", "Estoy muy feliz de estar aquí",
        "Tengo muchas razones para sonreír", "Siento una paz interior", "Todo es posible hoy",
        "Estoy rodeado de cosas buenas", "Este es un día memorable", "Estoy lleno de vida",
        "Hoy es un gran día para hacer cosas", "Qué maravilloso es vivir", "Siento que puedo lograr todo",
        "La vida es increíble", "Me siento afortunado", "Qué momento tan bonito", "Me siento lleno de amor",
        "Hoy todo es perfecto", "Estoy en paz", "El día no podría ser mejor", "Tengo una gran sonrisa en la cara",
        "Hoy nada puede salir mal", "El mundo está lleno de posibilidades", "Me siento en la cima del mundo",
        "Todo lo que hago hoy me trae alegría", "La vida es muy buena", "Todo lo que toco hoy es éxito",
        "No puedo dejar de sonreír", "Estoy en un estado de felicidad total", "Cada segundo de este día es perfecto",
        "Hoy todo es alegría", "Estoy muy agradecido por lo que tengo", "Todo está fluyendo de manera increíble",
        "Estoy muy emocionado por lo que viene", "Estoy disfrutando del día al máximo",
        "Todo lo que hago me llena de satisfacción", "Estoy extremadamente alegre",
        "La vida es un regalo y hoy lo siento plenamente",
        # 50 frases para "Melancólico"
        "Triste", "Desmotivado", "Deprimido", "Cansado", "Sin ganas",
        "Hoy me siento muy solo", "Todo parece tan difícil", "No veo salida a mis problemas",
        "Hoy no quiero hacer nada", "No tengo energías para seguir", "El día es gris, igual que mi ánimo",
        "Me siento derrotado", "Nada tiene sentido hoy", "Siento que no puedo avanzar",
        "Todo se siente pesado y agotador", "No hay nada que me haga sonreír hoy", "Siento una profunda tristeza",
        "No puedo dejar de pensar en lo mal que va todo", "Hoy no encuentro la luz en nada",
        "Todo parece ser en vano", "Hoy no tengo fuerzas para seguir", "Me siento atrapado",
        "Nada va bien en mi vida", "Me siento sin rumbo", "No tengo esperanza en nada",
        "Me siento desconectado de todo", "Todo me agobia", "Me siento solo en este mundo",
        "No sé cómo salir de este estado", "El futuro se ve oscuro", "Estoy roto por dentro",
        "No puedo dejar de sentirme mal", "Estoy en un pozo del que no puedo salir",
        "Hoy es un día sin sentido", "No encuentro motivos para seguir", "Todo parece una carga insostenible",
        "Me siento completamente vacío", "Nada me motiva hoy", "Siento una tristeza que no se va",
        "No hay nada que me anime", "Todo está mal hoy", "Me siento totalmente perdido",
        "No tengo fuerzas para seguir adelante", "Hoy todo es oscuridad", "Siento que todo está fuera de control",
        "No tengo ganas de nada", "Todo parece ser una carga", "Hoy no veo salida a mis problemas",
        "No encuentro nada positivo", "Nada tiene sentido en este día",
        # 50 frases para "Ni fu ni fa"
        "Indiferente", "Neutro", "Normal", "Regular", "Nada especial",
        "Hoy no siento nada en particular", "No es un mal día, pero tampoco es bueno",
        "Hoy no pasa nada interesante", "Me siento igual que siempre", "Nada nuevo hoy",
        "El día está bien, nada especial", "No me quejo, pero tampoco estoy feliz",
        "Hoy es un día como cualquier otro", "No me siento ni bien ni mal", "Todo sigue su curso normal",
        "Es solo otro día común", "No tengo grandes emociones hoy", "Hoy no tengo sentimientos fuertes",
        "El día es tranquilo, sin sobresaltos", "No tengo nada relevante que contar",
        "Hoy todo parece igual que siempre", "El día está pasando sin nada memorable",
        "Hoy es un día sin altibajos", "No ha pasado nada fuera de lo normal",
        "Me siento completamente neutro hoy", "Nada interesante ha sucedido", "Hoy es solo un día más",
        "No hay mucho que decir del día", "Todo está bien, sin más", "Hoy no ha pasado nada importante",
        "Hoy es un día sin grandes cambios", "Es un día tranquilo", "Todo parece estar en su lugar, nada más",
        "El día está bien, sin emociones fuertes", "No siento nada especial hoy",
        "Hoy me siento en un estado neutro", "No tengo ninguna emoción intensa",
        "Hoy es un día equilibrado, nada más", "Es un día más, sin grandes cosas que contar",
        "Todo ha estado normal hoy", "El día ha sido bastante tranquilo", "Hoy no ha sido ni bueno ni malo",
        "No siento nada fuerte hoy", "Es un día sin sobresaltos", "Hoy todo ha sido bastante regular",
        "No hay mucho que comentar del día", "Es un día como cualquier otro", "No ha pasado nada relevante hoy",
        "Hoy todo ha sido bastante normal", "Me siento en un estado intermedio"
    ],
    "estado_animo": [
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Alegre", "Alegre", "Alegre", "Alegre", "Alegre",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Melancólico", "Melancólico", "Melancólico", "Melancólico", "Melancólico",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa",
        "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa", "Ni fu ni fa"
    ]
}

# Convertir a DataFrame
df = pd.DataFrame(data)

### Transformamos las frases de entrenamiento del dataset con el transform de BERT, luego hacemos label encoder para transformar las etiquetas de los estados de animos en numericas 

In [6]:
embeddings_train = model.encode(df["frase"].tolist())

# Convertir las etiquetas de texto a numericas
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df["estado_animo"])

# Entrenar el modelo con Regresión Logística
clf = LogisticRegression(random_state=42, max_iter=200)
clf.fit(embeddings_train, y_train)

LogisticRegression(max_iter=200, random_state=42)

# Aplicacion

In [ ]:
class Recomendador:
    def __init__(self, modelo, clf, label_encoder, df_juegos_de_mesa, df_peliculas, df_libros):
        self.modelo = modelo
        self.clf = clf
        self.label_encoder = label_encoder
        self.df_juegos_de_mesa = df_juegos_de_mesa
        self.df_peliculas = df_peliculas
        self.df_libros = df_libros
    
    def solicitar_input(self):
        """
        Solicita la frase del usuario para clasificar el estado de ánimo.
        """
        frase_usuario = input("Ingresa una frase para clasificar tu estado de ánimo: ")
        print("La frase a clasificar es")
        return frase_usuario
    
    def procesar_estado_animo(self, frase_usuario):
        """
        Convierte la frase del usuario en embeddings y predice el estado de ánimo.
        """
        # Generar embeddings para la frase del usuario
        embedding_frase = self.modelo.encode([frase_usuario])

        # Predecir el estado de ánimo
        estado_animo_predicho = self.clf.predict(embedding_frase)

        # Convertir la predicción numérica de vuelta a texto
        estado_animo_predicho_texto = self.label_encoder.inverse_transform(estado_animo_predicho)
        
        print(f"Su estado de ánimo es: {estado_animo_predicho_texto[0]}")
        return estado_animo_predicho_texto[0]
    
    def solicitar_tematica(self):
        """
        Solicita la temática de interés del usuario.
        """
        tematica = input("Ingresa una temática que le gustaría explorar: ")
        return tematica

    def recomendar(self, input_estado_animo, estado_de_animo, input_tematica):
        """
        Genera recomendaciones basadas en el estado de ánimo, la temática y el modelo.
        """
        total = input_estado_animo + ' ' + estado_de_animo + ' ' + input_tematica
        print(f"Buscando recomendaciones para: {input_tematica}")
        
        # Vectorizamos la frase
        vector_comparar = self.modelo.encode(total)

        # Obtener los embeddings de cada dataset
        juegos_ubicacion = self.df_juegos_de_mesa['embeddings'].tolist()
        peliculas_ubicacion = self.df_peliculas['embeddings'].tolist()
        libros_ubicacion = self.df_libros['embeddings'].tolist()

        # Calcular similitudes
        similitudes_juegos = cosine_similarity([vector_comparar], juegos_ubicacion)
        similitudes_peliculas = cosine_similarity([vector_comparar], peliculas_ubicacion)
        similitudes_libros = cosine_similarity([vector_comparar], libros_ubicacion)

        # Obtener los índices de las top 3 recomendaciones
        top_indices_juegos = similitudes_juegos.argsort()[0][-3:][::-1]  # Top 3 juegos
        top_indices_peliculas = similitudes_peliculas.argsort()[0][-3:][::-1]  # Top 3 películas
        top_indices_libros = similitudes_libros.argsort()[0][-3:][::-1]  # Top 3 libros

        # Crear diccionarios con las recomendaciones
        recomendaciones_juegos = [
            {
                'titulo': self.df_juegos_de_mesa.iloc[index]['game_name'],
                'summary': self.df_juegos_de_mesa.iloc[index]['description']
            } for index in top_indices_juegos
        ]

        recomendaciones_peliculas = [
            {
                'titulo': self.df_peliculas.iloc[index]['Title'],
                'summary': self.df_peliculas.iloc[index]['Description']
            } for index in top_indices_peliculas
        ]

        recomendaciones_libros = [
            {
                'titulo': self.df_libros.iloc[index]['Title'],
                'summary': self.df_libros.iloc[index]['Summary']
            } for index in top_indices_libros
        ]

        # Devolver un diccionario con las recomendaciones
        return {
            'juegos': recomendaciones_juegos,
            'peliculas': recomendaciones_peliculas,
            'libros': recomendaciones_libros
        }

    def ejecutar(self):
        """
        Ejecuta el flujo completo de la aplicación.
        """
        frase_usuario = self.solicitar_input()
        estado_animo_predicho = self.procesar_estado_animo(frase_usuario)
        tematica = self.solicitar_tematica()

        # Generar recomendaciones
        recomendaciones = self.recomendar(frase_usuario, estado_animo_predicho, tematica)

        # Mostrar las recomendaciones
        print("\nRecomendaciones de juegos:")
        for rec in recomendaciones['juegos']:
            # print(f"- {rec['titulo']}: {rec['summary']}")
            print(f"- {rec['titulo']}")
        
        print("\nRecomendaciones de películas:")
        for rec in recomendaciones['peliculas']:
            print(f"- {rec['titulo']}: {rec['summary']}")
        
        print("\nRecomendaciones de libros:")
        for rec in recomendaciones['libros']:
            print(f"- {rec['titulo']}: {rec['summary']}")


In [8]:
recomendador = Recomendador(model, clf, label_encoder, df_juegos_de_mesa, df_peliculas, df_libros)

In [9]:
recomendador.ejecutar()

Su estado de ánimo es: Alegre
Buscando recomendaciones para: Alegre

Recomendaciones de juegos:
- Maria
- Azul: Queen's Garden
- Grand Austria Hotel

Recomendaciones de películas:
- Les Misérables: In 19th-century France, Jean Valjean, who for decades has been hunted by the ruthless policeman Javert after breaking parole, agrees to care for a factory worker's daughter. The decision changes their lives forever.
- The Promise: Set during the last days of the Ottoman Empire, The Promise follows a love triangle between Michael, a brilliant medical student, the beautiful and sophisticated Ana, and Chris - a renowned American journalist based in Paris.
- Woman in Gold: Maria Altmann, an octogenarian Jewish refugee, takes on the Austrian government to recover artwork she believes rightfully belongs to her family.

Recomendaciones de libros:
- The Anabasis of Alexanderor, The History of the Wars and Conquests of Alexander the Great: The Anabasis of Alexander" by Arrian is a historical account 